## Import libraries

In [36]:
import numpy as np
import pandas as pd
import scipy as sc
import scipy.fftpack
import pywt
import matplotlib.pyplot as plt
import cmath
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.pipeline import Pipeline

/Users/ilyas/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Clear Data

In [3]:
f_read = open("../data/WISDM/WISDM_ar_v1.1_raw.txt", "r")
f_write = open("../data/WISDM/WISDM_ar_v1.1_raw_cleared.txt", "w")
for string in f_read.readlines():
    if string[-1] == '\n':
        string = string[:-1]
    if len(string) > 0:
        string_list = string.split(';')
        if len(string_list) > 2:
            for row in string_list[:2]:
                words = row.split(',')
                if len(words) > 5:
                    if len(words[5]) > 0:
                        f_write.write("%s,%s,%s,%s,%s,%s\n" % (words[0], words[1], 
                                                               words[2], words[3], 
                                                               words[4], words[5]))
        else:
            words = string_list[0].split(',')
            if len(words) > 5:
                if len(words[5]) > 0:
                    f_write.write("%s,%s,%s,%s,%s,%s\n" % (words[0], words[1], 
                                                           words[2], words[3], 
                                                           words[4], words[5]))
f_read.close()
f_write.close()

## Read Data

In [4]:
data_wisdm = pd.read_table("../data/WISDM/WISDM_ar_v1.1_raw_cleared.txt", delimiter=',', header=None)
data_wisdm.columns = ['id_user', 'activity', 'timestamp', 'x', 'y', 'z']

## Segmentation

Let us construct 10 seconds time series segments.
* each time series should be from one user and one type of activity;
* in the time series timestamp shouldn't differ more then 0.2 second (empirical rule, in ideal all timestamp should differ on 50 ms = 0.05 second).

In [5]:
def check_candidate(candidate, data_type, threshold=2.*1e8):
    if data_type == "USCHAD":
        threshold = 0.
    tsp = np.array(candidate['timestamp'])
    diffs = tsp[1:] - tsp[:-1]
    
    return np.sum(diffs > threshold) == 0

def get_time_series(accelerations, data_type, nb=200):
    accelerations.index = [i for i in range(len(accelerations))]
    TS = []
    st = 0
    fi = st + nb
    while fi < len(accelerations):
        candidate = accelerations.loc[[st + i for i in range(nb)], :]
        if check_candidate(candidate, data_type):
            TS.append([np.array(candidate['x']), 
                       np.array(candidate['y']), 
                       np.array(candidate['z'])])
        st = fi
        fi += nb
    
    return TS

## Feature Generation

In [6]:
def get_distribution(data, df):
    classes = list(set(data['activity']))
    for activity in classes:
        nb = np.sum(df['activity'] == classes.index(activity))
        print("{:<20}{:<9d}{:<5.2f} %".format(activity, nb, 100. * nb / df.shape[0]))
    print("")
    print("Number of objects: {:d}".format(df.shape[0]))

In [7]:
def get_feature_matrix(data, data_type, get_feature_names, get_features, params=[]):
    
    classes = list(set(data['activity']))
    feature_names = get_feature_names(params)
    df = pd.DataFrame(columns=['activity']+feature_names) 

    id_range = np.unique(np.array(data['id_user']))
    for id_user in id_range:
        for activity in classes:
            mask = (data.loc[:, 'id_user'] == id_user) & (data.loc[:, 'activity'] == activity)
            accelerations = data.loc[mask, ['timestamp', 'x', 'y', 'z']].copy()
            TS = get_time_series(accelerations, data_type, nb=200)
            for ts in TS:
                features = get_features(ts, params)
                df.loc[len(df), :] = [classes.index(activity)] + features
    return df

## Feature extraction

### Expert functions

The idea is the following: we will consider 10 seconds time series (or 200 points of measurements) and calculate 40 features:
* ```[3]``` - mean acceleration of each axis;
* ```[3]``` - std of acceleration of each axis;
* ```[3]``` - mean absolute deviation of acceleration of each axis;
* ```[1]``` - mean acceleration;
* ```[30]``` - distribution of time series values of each axis. First of all we calculate min and max of each component ($X, Y, Z$) from the whole interval. Then we divide the range of values of each component into 10 equal intervals and calculate on each each interval the percent of values that are in it (in the corresponding interval).  

And apply LogisticRegression and SVM.

In [11]:
def get_expert_names(params):
    feature_names = ['avg_x', 'avg_y', 'avg_z', 
                     'std_x', 'std_y', 'std_z', 
                     'abs_x', 'abs_y', 'abs_z', 'mean']
    for i in range(10):
        name = str(i) + '_'
        feature_names += [name + 'x', name + 'y', name + 'z']
        
    return feature_names

def get_expert_features(ts, params):
    x = ts[0]
    y = ts[1]
    z = ts[2]
    n = x.shape[0]
    features = []
    features.append(x.mean())
    features.append(y.mean())
    features.append(z.mean())
    features.append(x.std())
    features.append(y.std())
    features.append(z.std())
    features.append(np.abs(x - x.mean()).mean())
    features.append(np.abs(y - y.mean()).mean())
    features.append(np.abs(z - z.mean()).mean())
    features.append((x+y+z).mean() / 3.)
    x_range = np.linspace(x.min(), x.max(), 11)
    y_range = np.linspace(y.min(), y.max(), 11)
    z_range = np.linspace(z.min(), z.max(), 11)
    for i in range(10):
        features.append(1. * np.sum((x_range[i] <= x) & (x < x_range[i+1])) / n)
        features.append(1. * np.sum((y_range[i] <= y) & (y < y_range[i+1])) / n)
        features.append(1. * np.sum((z_range[i] <= z) & (z < z_range[i+1])) / n)
    
    return features

Create and save:

In [12]:
df_expert_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_expert_names, get_expert_features)
df_expert_wisdm.to_csv("../data/features/expert_wisdm.csv", index=False)

In [26]:
#df_expert_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_expert_names, get_expert_features)
#df_expert_uschad.to_csv("../data/features/expert_uschad.csv", index=False)

### Autoregression model

### Explanation

In [13]:
def get_autoregressive_names(params):
    n = params[0]
    feature_names = []
    for ax in ['x', 'y', 'z']:
        feature_names += ['intercept_' + ax]
        for i in range(n):
            feature_names += ['coef_' + str(i) + '_' + ax]
            
    return feature_names

def get_autoregressive_features(ts, params):
    n = params[0]
    x = ts[0]
    y = ts[1]
    z = ts[2]
    m = x.shape[0]
    features = []
    X = np.zeros([m-n, n])
    Y = np.zeros(m-n)
    for axis in [x, y, z]:
        for i in range(m-n):
            X[i, :] = axis[i:i+n]
            Y[i] = axis[i+n]
        lr = LinearRegression()
        lr.fit(X, Y)
        features.append(lr.intercept_)
        features.extend(lr.coef_)
    
    return features

Create and save:

In [14]:
params = [20]
df_ar_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_autoregressive_names,
                                 get_autoregressive_features, params)
df_ar_wisdm.to_csv("../data/features/ar_wisdm.csv", index=False)

In [29]:
#params = [20]
#df_ar_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_autoregressive_names,
#                                  get_autoregressive_features, params)
#df_ar_uschad.to_csv("../data/features/ar_uschad.csv", index=False)

### Spectrum analysis

### Explanation

In [30]:
def get_spectrum_names(params):
    n = params[0]
    feature_names = []
    for ax in ['x', 'y', 'z']:
        for i in range(n):
            feature_names += ['eigv_' + str(i) + '_' + ax]
            
    return feature_names

def get_spectrum_features(ts, params):
    n = params[0]
    x = ts[0]
    y = ts[1]
    z = ts[2]
    m = x.shape[0]
    features = []
    X = np.zeros([m-n, n])
    Y = np.zeros(m-n)
    for axis in [x, y, z]:
        for i in range(m-n):
            X[i, :] = axis[i:i+n]
        h = sc.linalg.svd(X.T.dot(X), compute_uv=False, overwrite_a=True)
        features.extend(h)
    
    return features

Create and save:

In [31]:
params = [20]
df_ssa_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_spectrum_names,
                                  get_spectrum_features, params)
df_ssa_wisdm.to_csv("../data/features/ssa_wisdm.csv", index=False)

In [32]:
#params = [20]
#df_ssa_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_spectrum_names,
#                                   get_spectrum_features, params)
#df_ssa_uschad.to_csv("../data/features/ssa_uschad.csv", index=False)

## Fast Fourier Transform

### Explanation

In [20]:
def get_fft_names(params):
    n = params[0]
    feature_names = []
    for ax in ['x', 'y', 'z']:
        for i in range( 2 *n):
            feature_names += ['fft_coef_' + str(i) + '_' + ax]
            
    return feature_names

def get_fft_features(ts, params):
    n = params[0]
    x = ts[0]
    y = ts[1]
    z = ts[2]
    m = x.shape[0]
    features = []
    #X = np.zeros([m-n, n])
    #Y = np.zeros(m-n)
    for axis in [x, y, z]:
        h = sc.fftpack.fft(axis, n, axis=-1, overwrite_x=False)
        features.extend(polar(h))
        
    return features

def polar (lis):
    c = []
    for i in range(len(lis)):
        c.extend(cmath.polar(lis[i]))
        
    return c

In [21]:
params = [20]
df_fft_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_fft_names,
                                 get_fft_features, params)
df_fft_wisdm.to_csv("../data/features/fft_wisdm.csv", index=False)

/Users/ilyas/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/fftpack/basic.py:153: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[index]


In [22]:
df_fft_wisdm.to_csv("../data/features/fft_wisdm.csv", index=False)

# Wavelet transform

## Explanation

In [64]:
def get_wvt_names(params):
    n = params[0]
    feature_names = []
    for ax in ['x', 'y', 'z']:
        for i in range(2*n):
            feature_names += ['wvt_coef_' + str(i) + '_' + ax]
            
    return feature_names

def get_wvt_features(ts, params):
    n = params[0]
    x = ts[0]
    y = ts[1]
    z = ts[2]
    m = x.shape[0]
    features = []
    for axis in [x, y, z]:
        h = pywt.dwt(axis, 'db1')
        features.extend(h[0][:n])
        features.extend(h[1][:n])

    return features

In [65]:
params = [20]
df_wvt_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_wvt_names,
                                 get_wvt_features, params)
df_wvt_wisdm.to_csv("../data/features/wvt_wisdm.csv", index=False)

In [66]:
df_wvt_wisdm.to_csv("../data/features/wvt_wisdm.csv", index=False)

In [ ]:
df_wvt_wisdm.head(5)

# Classification

## Read object-feature matrix

In [68]:
data_wisdm = pd.read_table("../data/WISDM/WISDM_ar_v1.1_raw_cleared.txt", delimiter=',')
data_wisdm.columns = ['id_user', 'activity', 'timestamp', 'x', 'y', 'z']

#data_uschad = pd.read_table("../data/USC-HAD/USC-HAD_cleared.txt", delimiter=',')
#data_uschad.columns = ['id_user', 'activity', 'timestamp', 'x', 'y', 'z']

In [69]:
#df_expert_wisdm = pd.read_csv("../data/features/expert_wisdm.csv")
#df_ar_wisdm = pd.read_csv("../data/features/ar_wisdm.csv")
#df_ssa_wisdm = pd.read_csv("../data/features/ssa_wisdm.csv")
#df_fft_wisdm = pd.read_csv("../data/features/fft_wisdm.csv")
df_wvt_wisdm = pd.read_csv("../data/features/wvt_wisdm.csv")

In [70]:
df_wvt_wisdm.head(5)

,activity,wvt_coef_0_x,wvt_coef_1_x,wvt_coef_2_x,wvt_coef_3_x,wvt_coef_4_x,wvt_coef_5_x,wvt_coef_6_x,wvt_coef_7_x,wvt_coef_8_x,...,wvt_coef_30_z,wvt_coef_31_z,wvt_coef_32_z,wvt_coef_33_z,wvt_coef_34_z,wvt_coef_35_z,wvt_coef_36_z,wvt_coef_37_z,wvt_coef_38_z,wvt_coef_39_z
0,1,-0.028284,-4.900250,-0.268701,-3.556747,-4.362849,-0.417193,-6.576093,0.650538,0.459619,...,-0.777817,0.763675,-0.784889,-2.135462,-1.025305,3.273904,9.963135,-6.066976,0.601041,2.552655
1,1,-15.330075,-12.480435,-10.267190,-14.898740,-17.472609,-4.299209,-12.565288,-10.896515,-13.116831,...,-2.029396,2.283955,4.560839,-3.082986,1.654630,-8.789337,1.866762,-4.843681,7.078139,-2.736503
2,1,-11.971318,-15.598776,-12.006673,-6.505382,-7.368053,-9.941921,-16.362451,-18.908035,-11.780399,...,-7.035712,-0.056569,-6.519525,10.161124,-1.866762,0.374767,-0.565685,0.544472,-1.520280,6.936718
3,1,8.350931,-17.147339,-8.068088,-4.787113,1.647559,-6.116474,-5.663925,-1.944544,0.028284,...,1.407142,1.979899,2.764788,-2.283955,-2.630437,1.463711,1.796051,-10.479322,-2.333452,-6.816509
4,1,-13.328963,-11.730901,-1.110158,-12.324871,-20.831366,-5.798276,-12.183450,-4.546697,-13.986572,...,0.183848,-2.199102,6.095260,-3.167838,3.464823,-3.061772,0.240416,-1.845549,-3.061772,0.777817


In [71]:
def get_internal_score(clf, X, y, max_iter=11):
    nb = np.unique(y).shape[0]
    scores = np.zeros(nb+1)
    for j in range(max_iter):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        scores[0] += accuracy_score(y_test, y_predict)
        for i in range(nb):
            scores[i+1] += accuracy_score(1*(np.array(y_test) == i), 
                                          1*(np.array(y_predict) == i))
            
    return scores / max_iter

In [72]:
def get_score(df, estimator, params_grid, test_size=0.3):
    X = df.loc[:, df.columns != 'activity'].values
    y = df['activity'].values
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    clf = GridSearchCV(estimator, params_grid)
    clf.fit(X, list(y))
    clf_lr = clf.best_estimator_
    scores = get_internal_score(clf_lr, X, list(y))
    
    return scores

## Testing part 

In [73]:
parameters_lr = {'lr__penalty': ['l1', 'l2'], 
                 'lr__class_weight': ['balanced', None], 
                 'lr__C': 10. ** np.arange(-1, 4, 1)}
parameters_svm = {'svc__kernel': ['rbf'], 
                  'svc__C': 10. ** np.arange(-1, 4, 1), 
                  'svc__gamma': 10. ** np.arange(-3, 2, 1),
                  'svc__class_weight': ['balanced', None]}

parameters_rf = {'n_estimators': [200], 
                 'class_weight': ['balanced', None], 
                 'max_depth': [None, 3, 5, 11]}

scores_wisdm = {}
scores_uschad = {}

In [74]:
lr_pipeline = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression())])
svc_pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
rfc = RFC()

**Expert** features:

In [35]:
scores_wisdm['lr_expert'] = get_score(df_expert_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_expert'] = get_score(df_expert_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_expert'] = get_score(df_expert_wisdm, rfc, parameters_rf)

/Users/ilyas/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/ilyas/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/ilyas/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/ilyas/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/ilyas/anaconda2/envs/py36/lib/python3.6/site-packages/skl

KeyboardInterrupt: 

From **autoregression model** features:

In [36]:
n = 20

In [ ]:
scores_wisdm['lr_ar_' + str(n)] = get_score(df_ar_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_ar_' + str(n)] = get_score(df_ar_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_ar_' + str(n)] = get_score(df_ar_wisdm, rfc, parameters_rf)

From **spectrum analysis** features:

In [37]:
n = 20

In [38]:
scores_wisdm['lr_ssa_' + str(n)] = get_score(df_ssa_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_ssa_' + str(n)] = get_score(df_ssa_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_ssa_' + str(n)] = get_score(df_ssa_wisdm, rfc, parameters_rf)

NameError: name 'df_ssa_wisdm' is not defined

From **Fourier transform** features:  

In [30]:
scores_wisdm['lr_fft_'] = get_score(df_fft_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_fft_'] = get_score(df_fft_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_fft_'] = get_score(df_fft_wisdm, rfc, parameters_rf)

From **Wavelet transform** features:  

In [ ]:
scores_wisdm['lr_wvt_'] = get_score(df_wvt_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_wvt_'] = get_score(df_wvt_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_wvt_'] = get_score(df_wvt_wisdm, rfc, parameters_rf)

# Results

In [31]:
results_wisdm = pd.DataFrame.from_dict(scores_wisdm, orient='index').sort_index()
results_wisdm.columns = ['all'] + list(set(data_wisdm['activity']))
results_wisdm.to_csv("./results/results_wisdm_norm.csv")

In [32]:
results_wisdm

,all,Sitting,Downstairs,Standing,Upstairs,Jogging,Walking
lr_fft_,0.826383,0.994533,0.926474,0.992430,0.908320,0.970211,0.860798
rf_fft_,0.845938,0.996846,0.925562,0.995304,0.911474,0.977290,0.885400
svm_fft_,0.851616,0.996215,0.926544,0.994813,0.917011,0.984720,0.883928
